# Southeast Asian Android Phone Marketplace Analysis
## AI-Powered Analytics Dashboard

In [ ]:
# Install required packages
!pip install openai
!pip install git+https://github.com/openai/swarm.git

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from openai import OpenAI
from swarm import Swarm
from agents.base_agent import BaseAgent
from agents.data_quality_agent import DataQualityAgent
from agents.marketing_agent import MarketingAgent
from agents.customer_journey_agent import CustomerJourneyAgent
from agents.revenue_agent import RevenueAgent
from agents.visual_agent import VisualAgent
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Initialize OpenAI client
api = OpenAI(api_key="your-api-key-here")
client = Swarm(api)

# Load and preprocess data
df = pd.read_csv('ai first sales data - sales.csv')

# Initialize base agent for preprocessing
base_agent = BaseAgent("Base", "", [])
df_clean = base_agent.preprocess_data(df)

print(f"Dataset loaded with {len(df_clean)} rows")

In [ ]:
# Initialize all agents
agents = {
    'data_quality': DataQualityAgent(),
    'marketing': MarketingAgent(),
    'customer_journey': CustomerJourneyAgent(),
    'revenue': RevenueAgent(),
    'visual': VisualAgent()
}

### 1. Data Quality Analysis

In [ ]:
quality_response = client.run(
    agent=agents['data_quality'],
    messages=[{
        "role": "user",
        "content": "Analyze data quality and provide insights"
    }],
    context_variables={"dataframe": df_clean}
)

print("\nData Quality Analysis:")
if hasattr(quality_response, 'messages'):
    print(quality_response.messages[-1].content)
else:
    print(quality_response.get('content', str(quality_response)))

### 2. Marketing Analysis

In [ ]:
marketing_response = client.run(
    agent=agents['marketing'],
    messages=[{"role": "user", "content": "Analyze marketing performance"}],
    context_variables={"dataframe": df_clean}
)

print("\nMarketing Analysis:")
if hasattr(marketing_response, 'messages'):
    print(marketing_response.messages[-1].content)

# Marketing visualizations
source_metrics = df_clean.groupby('source').agg({
    'revenue': 'sum',
    'ad_spend': 'sum',
    'transactions': 'sum'
}).reset_index()

# Revenue by Channel
fig1 = px.pie(source_metrics, values='revenue', names='source',
              title='Revenue by Channel')
fig1.show()

# Revenue vs Ad Spend
fig2 = px.bar(source_metrics, x='source', y=['revenue', 'ad_spend'],
              title='Revenue vs Ad Spend by Channel')
fig2.show()

### 3. Customer Journey Analysis

In [ ]:
journey_response = client.run(
    agent=agents['customer_journey'],
    messages=[{"role": "user", "content": "Analyze customer journey"}],
    context_variables={"dataframe": df_clean}
)

print("\nCustomer Journey Analysis:")
if hasattr(journey_response, 'messages'):
    print(journey_response.messages[-1].content)

# Funnel visualization
funnel_data = {
    'Stage': ['Pageviews', 'Product Clicks', 'Add to Cart', 'Checkout', 'Transactions'],
    'Count': [
        df_clean['pageviews'].sum(),
        df_clean['productClick'].sum(),
        df_clean['addToCart'].sum(),
        df_clean['checkout'].sum(),
        df_clean['transactions'].sum()
    ]
}
fig = go.Figure(go.Funnel(y=funnel_data['Stage'], x=funnel_data['Count']))
fig.update_layout(title='Customer Journey Funnel')
fig.show()

### 4. Revenue Analysis

In [ ]:
revenue_response = client.run(
    agent=agents['revenue'],
    messages=[{"role": "user", "content": "Analyze revenue patterns"}],
    context_variables={"dataframe": df_clean}
)

print("\nRevenue Analysis:")
if hasattr(revenue_response, 'messages'):
    print(revenue_response.messages[-1].content)

# Revenue trend visualization
daily_revenue = df_clean.groupby('date')['revenue'].sum().reset_index()
fig = px.line(daily_revenue, x='date', y='revenue',
              title='Daily Revenue Trend')
fig.update_xaxes(type='date')
fig.show()

### 5. Visual Analysis

In [ ]:
visual_response = client.run(
    agent=agents['visual'],
    messages=[{"role": "user", "content": "Create visualizations"}],
    context_variables={"dataframe": df_clean}
)

if hasattr(visual_response, 'figures'):
    for fig in visual_response.figures:
        fig.show()